In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use("ggplot")
import pandas as pd
import random as rand
import string
import math
chars = string.ascii_uppercase + string.digits
%matplotlib inline
plt.rcParams["figure.figsize"] = (16,8)
import seaborn as sns
import sklearn as skl
import skimage as ski
from sklearn.preprocessing import LabelEncoder
from scipy import spatial
from sklearn.utils import shuffle
from os import system
import skimage as ski
from skimage.color import rgb2gray
from scipy import misc
from skimage.exposure import adjust_gamma
from sklearn import preprocessing
from scipy import stats
from glob import glob

In [2]:
gene_list = pd.read_csv('allDiffexpGenes.csv')

In [3]:
print(gene_list.shape)
gene_list.head()

(356, 5)


,gene,qval,up_or_down,gene_name,description
0,FBgn0003071,0.000089,up,Pfk,Catalyzes the phosphorylation of D-fructose 6-...
1,FBgn0028426,0.000458,down,JhI-1,Juvenile hormone-inducible protein 1 is a high...
2,FBgn0013679,0.001081,up,mt:ND1,Core subunit of the mitochondrial membrane res...
3,FBgn0013680,0.001081,up,mt:ND2,The NADH:ubiquinone oxidoreductase complex (Mi...
4,FBgn0013681,0.001081,up,mt:ND3,oxidation-reduction process.


In [4]:
gene_list_upregulated = gene_list[gene_list["up_or_down"] == 'up']
gene_list_downregulated = gene_list[gene_list["up_or_down"] == 'down']

In [5]:
gene_list_downregulated.shape

(192, 5)

In [6]:
gene_list_upregulated.shape

(164, 5)

In [7]:
gene_names_upregulated = gene_list_upregulated.gene_name

In [8]:
gene_names_upregulated.to_csv('Upregulated genes names.csv', index = False)
gene_list_upregulated.gene.to_csv('Upregulated genes code.csv', index = False)

In [9]:
gene_names_downregulated = gene_list_downregulated.gene_name
gene_names_downregulated.to_csv('Downregulated genes names.csv', index = False)
gene_list_downregulated.gene.to_csv('Downregulated genes code.csv', index = False)

# used table browser from UCSC to obtain the bed files for upregulated and downregulated genes 

https://ampliseq.com/help/bedFilesUcsc.action

it dropped a few genes for some reason

then made a image of the chromossomes positions using https://www.ncbi.nlm.nih.gov/genome/tools/gdp

for some weird reason the number of genes jumped from ~180 to 381 in both cases. Need to understand why that happened

Changed the header of the bed file cause it's a mess and lacked the column names

In [12]:
bed_upregal = pd.read_csv('upregul_bed.csv', sep= "	")
print(bed_upregal.shape)
bed_upregal.sort_values('gene').head()

    

(381, 12)


,chromosome,start,stop,gene,score,strand,thickStart,thickEnd,itemRgb,blockCount,blockSizes,blockStarts
74,chr2L,7018086,7023898,NM_001014477,0,-,7018144,7023705,0,4,"288,650,248,370,","0,344,1048,5442,"
122,chr2R,24990947,25011463,NM_001014552,0,-,24991136,25011341,0,13,"325,164,1296,531,305,2740,93,24,122,157,216,23...","0,862,1085,2443,3064,3459,6261,7330,9091,11340..."
123,chr2R,24990947,25011463,NM_001014553,0,-,24991136,25010511,0,13,"325,164,1296,531,305,2740,93,24,122,157,216,23...","0,862,1085,2443,3064,3459,6261,7330,9091,11340..."
293,chr3R,25983012,25995737,NM_001014668,0,+,25983291,25994574,0,18,"336,144,385,142,594,322,553,538,1592,607,2356,...","0,579,822,1262,1460,2115,2499,3106,3699,5347,6..."
18,chrX,16361568,16368526,NM_001014747,0,+,16361697,16365744,0,4,"486,1273,309,2940,","0,1789,3636,4018,"


In [13]:
bed_downregal = pd.read_csv('downregul_bed.csv', sep= "	")
print(bed_upregal.shape)
bed_downregal.sort_values('gene').head()

(381, 12)


,chromosome,start,stop,gene,score,strand,thickStart,thickEnd,itemRgb,blockCount,blockSizes,blockStarts
529,chr2R,24903415,24903935,NM_001014551,0,-,24903722,24903892,0,3,"174,91,55,","0,282,465,"
50,chr2L,21419530,21419943,NM_001032113,0,-,21419575,21419887,0,1,"413,","0,"
53,chr2L,21439588,21440001,NM_001032113,0,-,21439633,21439945,0,1,"413,","0,"
54,chr2L,21453789,21454202,NM_001032113,0,-,21453834,21454146,0,1,"413,","0,"
55,chr2L,21484084,21484497,NM_001032113,0,-,21484129,21484441,0,1,"413,","0,"


In [14]:
bed_upregal.iloc[:, :3].to_csv('upregulated_Chr_start_Stop.csv', index = False)


In [15]:
bed_downregal.iloc[:, :3].to_csv('downregulated_Chr_start_Stop.csv', index= False)

# opening file from rVista 

http://genome.lbl.gov/vista/downloads.shtml

dmel_dvir.gz	- transcription factor binding sites in promoters (5K upstream)
                      of D.melanogaster Release 5 FlyBase genes conserved
                      in D.melanogaster-D.virilis alignments
                      
                    

In [19]:
tf_binding = pd.read_csv('bed files/dmel_dvir',header = None, sep= "	")

In [38]:
print(tf_binding.shape)
tf_binding.head()

(2018959, 6)


,0,1,2,3,4,5
0,FBgn0000053,Abd-B,chr2L,7024253,7024259,+
1,FBgn0000053,Abd-B,chr2L,7024257,7024263,+
2,FBgn0000053,Abd-B,chr2L,7024916,7024922,+
3,FBgn0000053,Abd-B,chr2L,7025928,7025934,-
4,FBgn0000053,Abd-B,chr2L,7025931,7025937,+


# find bindings for upregulated and downregulated genes

In [36]:
tf_down = tf_binding[tf_binding[0].isin(gene_list_downregulated.gene.values)]
tf_down.shape

(20476, 6)

In [39]:
tf_up = tf_binding[tf_binding[0].isin(gene_list_upregulated.gene.values)]
tf_up.shape

(19743, 6)

# Find unique TFs and count their frequencies

In [82]:
tf_down_names = tf_down.groupby(1)[0].nunique()
tf_down_names.to_csv('tf genes binding to downregulated genes')
tf_down_namesT = pd.read_csv('tf genes binding to downregulated genes', header = None)
tf_down_namesT.sort_values(1, ascending=False)

,0,1
8,CF2-II,79
3,BR-C,79
4,BYN,79
7,CF1A,79
2,Antp,78
29,TWI,77
27,Sn,77
20,Hairy,77
21,Hb,76
0,Abd-B,75


In [77]:
tf_up_names = tf_up.groupby(1)[0].nunique()
tf_up_names.to_csv('tf genes binding to upregulated genes')
tf_up_namesT = pd.read_csv('tf genes binding to upregulated genes', header = None)
tf_up_namesT.sort_values(1, ascending=False)

,0,1
8,CF2-II,71
3,BR-C,71
4,BYN,71
7,CF1A,70
0,Abd-B,69
29,TWI,69
2,Antp,68
27,Sn,68
21,Hb,67
20,Hairy,63


(34, 2)